# Query Top-N
The following example show a way to extract with the ampligraph.discovery.query_topn API the top N candidates to be true positives.

In [1]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
import requests
from ampligraph.datasets import load_from_csv
# Game of Thrones relations dataset
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv'
open('GoT.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'GoT.csv', sep=',')

In [3]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel

# Create, compile and fit the model
model = ScoringBasedEmbeddingModel(eta=5, 
                                   k=150,
                                   scoring_type='DistMult')



model.compile(optimizer='adam', loss='pairwise')
model.fit(X,
          batch_size=100,
          epochs=20, 
          verbose=False)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



In [4]:
from ampligraph.discovery import query_topn

query_topn(model, top_n=5,
           head='Eddard Stark', relation='ALLIED_WITH', tail=None,
           ents_to_consider=None, rels_to_consider=None)


(array([['Eddard Stark', 'ALLIED_WITH', 'House Stark of Winterfell'],
        ['Eddard Stark', 'ALLIED_WITH', 'House Westerling of the Crag'],
        ['Eddard Stark', 'ALLIED_WITH', "House Baratheon of Storm's End"],
        ['Eddard Stark', 'ALLIED_WITH', 'House Frey of the Crossing'],
        ['Eddard Stark', 'ALLIED_WITH', 'House Karstark of Karhold']],
       dtype='<U44'),
 array([2.4362302 , 0.6104515 , 0.5265138 , 0.5220847 , 0.39194268],
       dtype=float32))